In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import random
from selenium import webdriver
import datetime
import time
import numpy as np
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys 
from urllib.parse import quote_plus
from urllib.request import urlopen
import os

In [4]:
def Driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    #options.add_argument("headless")
    #options.add_argument('--start-fullscreen')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(options = options)
    return wd

In [5]:
# 베이스 URL과 상업적 이용 가능 옵션
def make_url(word):
    base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=image&query='
    ccl = '&res_fr=0&res_to=0&sm=tab_opt&color=&ccl=2&nso=so%3Ar%2Ca%3Aall%2Cp%3Aall&recent=0&datetype=0&startdate=0&enddate=0&gif=0&optStr=&nso_open=1&pq='
    return base_url + word + ccl

In [6]:
# 해당하는 폴더가 없을 경우 생성해주는 함수
def makedirs(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise

In [7]:
# 찾을 목록이 담긴 dictionary
search_dict = {
        '건강식단': ['샐러드','고구마요리','닭가슴살음식','연어음식','오트밀'],
        '밥류' : ['볶음밥','덮밥','한정식','돈까스','오므라이스'],
        '면류' : ['라면','짜장면','짬뽕','파스타','냉면', '국수'],
        '패스트푸드' : ['햄버거','감자튀김','피자','치킨'],
        '디저트류' : ['카페음료','와플','아이스크림','탄산음료','조각케이크','쿠키','빙수']}

In [45]:
def save_images(image_url, paths, file_name, i):
    import base64
        
    if 'data:' in str(image_url):
        image_url = str(image_url).replace('data:image/gif;base64,','')
        print(image_url)
        
        with open(os.path.join(paths,  file_name + '_' + str(i)) + '.jpg', 'wb') as f:
            f.write(base64.b64decode(image_url))
        print('save complete')
    else:
        t= urlopen(image_url).read()
        file = open(os.path.join(paths, file_name+'_'+str(i)+".jpg"), 'wb')
        file.write(t)

In [46]:
def naver_crawl(image_numbers):
    wd = Driver()
    wd.implicitly_wait(3)
    for meal in search_dict:
        for food in search_dict[meal]:
                # 음식에 해당하는 검색어를 입력한 페이지 출력
                wd.get(make_url(food))
                time.sleep(2)
                for i in range(1,image_numbers+1):
                    time.sleep(2)
                    # i에 해당하는 이미지가 없을 경우 PASS
                    try:
                        # image url 추출
                        images= wd.find_elements(By.XPATH, f'//*[@id="main_pack"]/section[2]/div/div[1]/div[1]/div[{i}]/div/div[1]/a/img')
                        save_path = f'/Users/mgd81/CP1/images/{meal}/{food}/'
                        makedirs(save_path)
                        src = images[0].get_attribute('src')
                        save_images(str(src), save_path, f'{meal}_{food}_', i)
                        print(src)

                        # 이미지가 10개가 넘어갈때 마다 PAGE_DOWN
                        if i % 10 == 0:
                            body = wd.find_element(By.XPATH,'//body').send_keys(Keys.PAGE_DOWN)
                            time.sleep(3)
                    except:
                        print(f"No element in {i}")
                        continue
    wd.close()
    print("End_crawling")

In [47]:
naver_crawl(10)
#//*[@id="main_pack"]/section[2]/div/div[1]/div[1]/div[5]/div/div[1]/a/img

https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjA5MjlfMjY4%2FMDAxNjY0NDYyMDExMTU0.HSyEZIsYCIQhRQPYZ2FzVEpEOTY5bW0OtROFsYFIpZcg.ECqwEkpI7_jYgDXA_gJXmzhC3PkgNW7x3hN160d1hn0g.JPEG.i95kitty%2F1664461999726.jpg&type=a340
https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjA5MjZfMTM2%2FMDAxNjY0MTU5NDkyMDQ0.W7AiJFG4_gUqoDwooQpRaFRkS1ydHXiACZRgehx_xTog.JkKeWrE4Ae3dZpm_krrT2h_7rZsxJDU12k53gTP8jL0g.JPEG.20000950%2Foutput_3437907188.jpg&type=l340_165
https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjA1MTNfOTUg%2FMDAxNjUyNDMzNjU0MzIz.4wnyz81jWVLl6oXl-Jwch2Qnny3lr3i73p7dl_wi0AYg.KElSAJdik8PQFzZPyuyE4x2Uc_IrSdScG4OD6Y7Iw00g.JPEG.naman_sarang%2F20220512%25A3%25DF181806.jpg&type=a340
https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjA3MzBfMTA2%2FMDAxNjU5MTUyMjUzMTQ0.5K4eWQ1MGeMNpr61m1oe1CnKr2Le5TcT63miCGDOQLkg.3hn7GL2EdsHMVkHbqGJN8D3iiChX2gftUqkQvymVzAUg.JPEG.uj0155%2F20220730%25A3%2

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x0102ACD3+2075859]
	Ordinal0 [0x00FBEE61+1633889]
	Ordinal0 [0x00EBB7BD+571325]
	Ordinal0 [0x00EA2E1A+470554]
	Ordinal0 [0x00F0AA0B+895499]
	Ordinal0 [0x00F1AC96+961686]
	Ordinal0 [0x00F07136+880950]
	Ordinal0 [0x00EDFEFD+720637]
	Ordinal0 [0x00EE0F3F+724799]
	GetHandleVerifier [0x012DEED2+2769538]
	GetHandleVerifier [0x012D0D95+2711877]
	GetHandleVerifier [0x010BA03A+521194]
	GetHandleVerifier [0x010B8DA0+516432]
	Ordinal0 [0x00FC682C+1665068]
	Ordinal0 [0x00FCB128+1683752]
	Ordinal0 [0x00FCB215+1683989]
	Ordinal0 [0x00FD6484+1729668]
	BaseThreadInitThunk [0x76916739+25]
	RtlGetFullPathName_UEx [0x77B28FD2+1218]
	RtlGetFullPathName_UEx [0x77B28F9D+1165]
